In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 49 kB/s 
     |████████████████████████████████| 199 kB 44.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=980e56b49a96f7f6e6fc88c37860eaa306558c7b9dc0f3596e3a51bbf5627858
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [39]:
# importar bibliotecas
import pyspark
from pyspark.sql import SparkSession

#Criação de uma sessão Spark(caso ela não exista) ou recuperação de uma 
#sessão existente com o mesmo nome
spark = SparkSession.builder.appName("Censo Escolar").master('local[*]').getOrCreate()

# criação do contexto

sc = spark.sparkContext

rdd = sc.textFile("censo_escolar_2021.csv")
rdd.take(4)

['NU_ANO_CENSO;NO_REGIAO;CO_REGIAO;NO_UF;SG_UF;CO_UF;NO_MUNICIPIO;CO_MUNICIPIO;NO_MESORREGIAO;CO_MESORREGIAO;NO_MICRORREGIAO;CO_MICRORREGIAO;CO_DISTRITO;CO_ENTIDADE;NO_ENTIDADE;TP_DEPENDENCIA;TP_CATEGORIA_ESCOLA_PRIVADA;TP_LOCALIZACAO;TP_LOCALIZACAO_DIFERENCIADA;DS_ENDERECO;NU_ENDERECO;DS_COMPLEMENTO;NO_BAIRRO;CO_CEP;NU_DDD;NU_TELEFONE;TP_SITUACAO_FUNCIONAMENTO;CO_ORGAO_REGIONAL;DT_ANO_LETIVO_INICIO;DT_ANO_LETIVO_TERMINO;IN_VINCULO_SECRETARIA_EDUCACAO;IN_VINCULO_SEGURANCA_PUBLICA;IN_VINCULO_SECRETARIA_SAUDE;IN_VINCULO_OUTRO_ORGAO;IN_CONVENIADA_PP;TP_CONVENIO_PODER_PUBLICO;IN_MANT_ESCOLA_PRIVADA_EMP;IN_MANT_ESCOLA_PRIVADA_ONG;IN_MANT_ESCOLA_PRIVADA_OSCIP;IN_MANT_ESCOLA_PRIV_ONG_OSCIP;IN_MANT_ESCOLA_PRIVADA_SIND;IN_MANT_ESCOLA_PRIVADA_SIST_S;IN_MANT_ESCOLA_PRIVADA_S_FINS;NU_CNPJ_ESCOLA_PRIVADA;NU_CNPJ_MANTENEDORA;TP_REGULAMENTACAO;TP_RESPONSAVEL_REGULAMENTACAO;CO_ESCOLA_SEDE_VINCULADA;CO_IES_OFERTANTE;IN_LOCAL_FUNC_PREDIO_ESCOLAR;TP_OCUPACAO_PREDIO_ESCOLAR;IN_LOCAL_FUNC_SALAS_EMPRESA;IN_

In [ ]:
# Ignorar o Cabeçalho

rdd = rdd.filter(lambda x:x.split(";")[0]!="NU_ANO_CENSO")
rdd.collect()

## **Resolução Questão 1**

In [48]:
# Questão 1 (Primeira Maneira para realizar a Questão - Mais elaborada)

rdd_map = rdd.map(lambda x: (x.split(";")[6], 1))
rdd_map.take(4)

[("Alta Floresta D'Oeste", 1),
 ("Alta Floresta D'Oeste", 1),
 ("Alta Floresta D'Oeste", 1),
 ("Alta Floresta D'Oeste", 1)]

In [49]:
rdd_reduce = rdd_map.reduceByKey(lambda x,y : x+y)
rdd_reduce.take(4)

[('Colorado do Oeste', 15),
 ('Costa Marques', 15),
 ("Nova Brasil�ndia D'Oeste", 11),
 ("Alvorada D'Oeste", 20)]

In [50]:
rdd_curitiba = rdd_reduce.filter(lambda x: x[0] == 'Curitiba')
rdd_curitiba.take(5)

[('Curitiba', 1181)]

In [51]:
rdd_curitiba.coalesce(1).saveAsTextFile("Escolas-Curitiba.txt")

In [ ]:
# Questão 1 (Segunda Maneira para realizar a Questão - Mais simplificada)

rddAlternativo = rdd.filter(lambda x:x.split(";")[6] == "Curitiba")
rddAlternativo.take(4)

In [ ]:
rddAlternativo.count()

1181

## **Resolução Questão 2**

In [46]:
rdd_rg_map = rdd.map(lambda x:(x.split(";")[1], 1))
rdd_rg_map.take(10)

[('Norte', 1),
 ('Norte', 1),
 ('Norte', 1),
 ('Norte', 1),
 ('Norte', 1),
 ('Norte', 1),
 ('Norte', 1),
 ('Norte', 1),
 ('Norte', 1),
 ('Norte', 1)]

In [42]:
rdd_rg_reduce = rdd_rg_map.reduceByKey(lambda x,y : x+y)
rdd_rg_reduce.collect()

[('Norte', 26095),
 ('Sul', 29018),
 ('Centro-Oeste', 11659),
 ('Nordeste', 79039),
 ('Sudeste', 75329)]

In [44]:
rdd_rg_ordenado = rdd_rg_reduce.sortBy(lambda x:x[0], ascending=True)
rdd_rg_ordenado.collect()

[('Centro-Oeste', 11659),
 ('Nordeste', 79039),
 ('Norte', 26095),
 ('Sudeste', 75329),
 ('Sul', 29018)]

In [ ]:
rdd_rg_reduce.coalesce(1).saveAsTextFile("Numero-Escolas-Região_Geográfica.txt")

# **Resolução Questão 3**

In [26]:
rdd_q3_map = rdd.map(lambda x:((x.split(";")[14], x.split(";")[6]) , x.split(";")[338] + x.split(";")[342] + x.split(";")[345]))
rdd_q3_map.take(10)

[(('EIEEF HAP BITT TUPARI', "Alta Floresta D'Oeste"), '110'),
 (('CEEJA LUIZ VAZ DE CAMOES', "Alta Floresta D'Oeste"), '1400'),
 (('EMMEF 7 DE SETEMBRO', "Alta Floresta D'Oeste"), '110'),
 (('EMEIEF ANA NERY', "Alta Floresta D'Oeste"), '1090'),
 (('EMEIEF BOA ESPERANCA', "Alta Floresta D'Oeste"), '15140'),
 (('EEEFM EURIDICE LOPES PEDROSO', "Alta Floresta D'Oeste"), '511640'),
 (('EMEIEF IZIDORO STEDILE', "Alta Floresta D'Oeste"), '15140'),
 (('EMEIEF JOSE BASILIO DA GAMA', "Alta Floresta D'Oeste"), '11110'),
 (('EEEMTI JUSCELINO KUBITSCHEK DE OLIVEIRA', "Alta Floresta D'Oeste"),
  '17917'),
 (('EMMEF MALBA TAHAN', "Alta Floresta D'Oeste"), '110')]

In [21]:
rdd_q3_ordenado = rdd_q3_map.sortBy(lambda x: x[1], ascending=False)
rdd_q3_ordenado.take(10)

[(('EEEFM PIONEIRA', 'Teixeir�polis'), '999'),
 (('ESCOLA MUNICIPAL INDIGENA VOVO JANDICO DA SILVA', 'Boa Vista'), '999'),
 (('ESC JOSE FRANCISCO DA SILVA', 'Pl�cido de Castro'), '998'),
 (('ESCOLA MUNICIPAL PROF� MARIA HELENA ALVES ARAUJO', 'Arraias'), '998'),
 (('ESC MUL TIRADENTES', 'Axix� do Tocantins'), '998'),
 (('ESCOLA MUNICIPAL SIMAO ALVES DE MOURA', 'Bernardo Say�o'), '998'),
 (('ESCOLA MUNICIPAL JOAQUIM ARAUJO', 'Porto Alegre do Tocantins'), '998'),
 (('UNIDADE INTEGRADA DE EDUCACAO ESCOLAR INDIGENA KATUAHY ZEMU E HAW',
   'Amarante do Maranh�o'),
  '998'),
 (('CENTRO EDUCACIONAL COLOMBO LADISLAU', 'Manaus'), '997132'),
 (('EMEF MARIO COVAS', 'Candeias do Jamari'), '997')]

## **Resolução Questão 4**

In [52]:
pairRDD_quantidade_regiao = rdd.map(lambda x:(x.split(";")[1], (int(x.split(";")[305]) if x.split(";")[305].isdigit() else 0, 1)))
pairRDD_quantidade_regiao.collect()



[('Norte', (0, 1)),
 ('Norte', (0, 1)),
 ('Norte', (0, 1)),
 ('Norte', (0, 1)),
 ('Norte', (0, 1)),
 ('Norte', (327, 1)),
 ('Norte', (0, 1)),
 ('Norte', (0, 1)),
 ('Norte', (190, 1)),
 ('Norte', (0, 1)),
 ('Norte', (0, 1)),
 ('Norte', (0, 1)),
 ('Norte', (0, 1)),
 ('Norte', (0, 1)),
 ('Norte', (0, 1)),
 ('Norte', (0, 1)),
 ('Norte', (0, 1)),
 ('Norte', (323, 1)),
 ('Norte', (0, 1)),
 ('Norte', (0, 1)),
 ('Norte', (15, 1)),
 ('Norte', (0, 1)),
 ('Norte', (12, 1)),
 ('Norte', (0, 1)),
 ('Norte', (0, 1)),
 ('Norte', (0, 1)),
 ('Norte', (0, 1)),
 ('Norte', (22, 1)),
 ('Norte', (0, 1)),
 ('Norte', (0, 1)),
 ('Norte', (0, 1)),
 ('Norte', (0, 1)),
 ('Norte', (0, 1)),
 ('Norte', (0, 1)),
 ('Norte', (0, 1)),
 ('Norte', (0, 1)),
 ('Norte', (0, 1)),
 ('Norte', (0, 1)),
 ('Norte', (0, 1)),
 ('Norte', (212, 1)),
 ('Norte', (0, 1)),
 ('Norte', (0, 1)),
 ('Norte', (832, 1)),
 ('Norte', (0, 1)),
 ('Norte', (0, 1)),
 ('Norte', (0, 1)),
 ('Norte', (255, 1)),
 ('Norte', (0, 1)),
 ('Norte', (0, 1)),
 ('No

In [53]:
pairRDD_soma_q4 = pairRDD_quantidade_regiao.reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1]))
pairRDD_soma_q4.collect()

[('Norte', (842008, 26095)),
 ('Sul', (991560, 29018)),
 ('Centro-Oeste', (643082, 11659)),
 ('Nordeste', (2246336, 79039)),
 ('Sudeste', (3047571, 75329))]

In [54]:
pairRDD_media = pairRDD_soma_q4.mapValues(lambda x: x[0]/x[1])
pairRDD_media.collect()

[('Norte', 32.26702433416363),
 ('Sul', 34.170514852849955),
 ('Centro-Oeste', 55.15756068273437),
 ('Nordeste', 28.420602487379647),
 ('Sudeste', 40.45680946249121)]

In [55]:
pairRDD_media.coalesce(1).saveAsTextFile("Exercicio4.txt")

## **Resolução Questão 5**

In [56]:
#PairRdd que realiza um Map que retorna uma nova lista com os valores dos índices (Localização e Dependência)
pairRdd_values = rdd.map(lambda x: ((x.split(";")[17], x.split(";")[15]), 1))
pairRdd_values.take(10)

[(('2', '2'), 1),
 (('1', '2'), 1),
 (('2', '3'), 1),
 (('1', '3'), 1),
 (('2', '3'), 1),
 (('1', '2'), 1),
 (('1', '3'), 1),
 (('2', '3'), 1),
 (('1', '2'), 1),
 (('2', '3'), 1)]

In [57]:
#somar as ocorrencias das chaves, apresentando a quantidade de escolas. 
#Por tipo de localização e Dependência
contagem_dependencia_local = pairRdd_values.reduceByKey(lambda x,y: x+y)
contagem_dependencia_local.collect()

[(('1', '1'), 619),
 (('1', '2'), 26796),
 (('1', '3'), 64278),
 (('2', '4'), 934),
 (('2', '2'), 6768),
 (('2', '3'), 70091),
 (('1', '4'), 51559),
 (('2', '1'), 95)]

In [58]:
#Criação do arquivo TXT com o resultado da função
contagem_dependencia_local.coalesce(1).saveAsTextFile("Escolas_local_Dependencia.txt")

## **Resolução Questão 6**

In [59]:
pairRDD_quant = rdd.map(lambda x: ((x.split(";")[4], x.split(";")[15]), 1))
pairRDD_quant.collect()

[(('RO', '2'), 1),
 (('RO', '2'), 1),
 (('RO', '3'), 1),
 (('RO', '3'), 1),
 (('RO', '3'), 1),
 (('RO', '2'), 1),
 (('RO', '3'), 1),
 (('RO', '3'), 1),
 (('RO', '2'), 1),
 (('RO', '3'), 1),
 (('RO', '3'), 1),
 (('RO', '3'), 1),
 (('RO', '3'), 1),
 (('RO', '3'), 1),
 (('RO', '3'), 1),
 (('RO', '3'), 1),
 (('RO', '2'), 1),
 (('RO', '2'), 1),
 (('RO', '3'), 1),
 (('RO', '2'), 1),
 (('RO', '2'), 1),
 (('RO', '2'), 1),
 (('RO', '2'), 1),
 (('RO', '2'), 1),
 (('RO', '3'), 1),
 (('RO', '2'), 1),
 (('RO', '2'), 1),
 (('RO', '2'), 1),
 (('RO', '2'), 1),
 (('RO', '4'), 1),
 (('RO', '3'), 1),
 (('RO', '3'), 1),
 (('RO', '3'), 1),
 (('RO', '3'), 1),
 (('RO', '2'), 1),
 (('RO', '2'), 1),
 (('RO', '4'), 1),
 (('RO', '2'), 1),
 (('RO', '4'), 1),
 (('RO', '2'), 1),
 (('RO', '2'), 1),
 (('RO', '3'), 1),
 (('RO', '2'), 1),
 (('RO', '3'), 1),
 (('RO', '4'), 1),
 (('RO', '3'), 1),
 (('RO', '2'), 1),
 (('RO', '3'), 1),
 (('RO', '3'), 1),
 (('RO', '3'), 1),
 (('RO', '2'), 1),
 (('RO', '3'), 1),
 (('RO', '3'

In [60]:
rdd_somaQuant = pairRDD_quant.reduceByKey(lambda x,y: x+y)
rdd_somaQuant.collect()

[(('AM', '2'), 786),
 (('PA', '3'), 10371),
 (('AP', '1'), 6),
 (('TO', '1'), 11),
 (('MA', '3'), 11432),
 (('MA', '2'), 1305),
 (('PI', '2'), 775),
 (('RN', '1'), 29),
 (('AL', '2'), 325),
 (('AL', '3'), 2474),
 (('SE', '3'), 1572),
 (('BA', '4'), 3297),
 (('ES', '1'), 23),
 (('RJ', '4'), 5301),
 (('SP', '4'), 13550),
 (('SP', '1'), 45),
 (('PR', '1'), 29),
 (('SC', '4'), 1608),
 (('MT', '3'), 1823),
 (('MT', '2'), 824),
 (('GO', '2'), 1040),
 (('DF', '4'), 704),
 (('DF', '1'), 11),
 (('AM', '4'), 366),
 (('TO', '3'), 1534),
 (('MA', '4'), 1394),
 (('PI', '1'), 24),
 (('CE', '2'), 761),
 (('RN', '3'), 3435),
 (('PB', '4'), 1022),
 (('SE', '4'), 625),
 (('BA', '2'), 1168),
 (('MG', '2'), 4319),
 (('ES', '3'), 2752),
 (('ES', '2'), 610),
 (('SP', '2'), 6572),
 (('SC', '3'), 4290),
 (('RS', '1'), 50),
 (('MS', '1'), 11),
 (('MT', '4'), 545),
 (('GO', '1'), 27),
 (('DF', '2'), 692),
 (('AC', '4'), 49),
 (('AC', '1'), 7),
 (('AM', '3'), 5081),
 (('RR', '3'), 468),
 (('CE', '1'), 34),
 (('P

In [61]:
rdd_somaQuant_ordenado = rdd_somaQuant.sortBy(lambda x:x[1] , ascending=False)
rdd_somaQuant_ordenado.collect()

[(('BA', '3'), 15659),
 (('SP', '3'), 13822),
 (('SP', '4'), 13550),
 (('MG', '3'), 13363),
 (('MA', '3'), 11432),
 (('PA', '3'), 10371),
 (('CE', '3'), 7255),
 (('SP', '2'), 6572),
 (('RJ', '3'), 6368),
 (('MG', '4'), 6259),
 (('PE', '3'), 6184),
 (('RS', '3'), 5867),
 (('PR', '3'), 5487),
 (('RJ', '4'), 5301),
 (('PI', '3'), 5281),
 (('AM', '3'), 5081),
 (('MG', '2'), 4319),
 (('SC', '3'), 4290),
 (('PB', '3'), 3895),
 (('RN', '3'), 3435),
 (('BA', '4'), 3297),
 (('PE', '4'), 3216),
 (('RS', '4'), 3165),
 (('ES', '3'), 2752),
 (('GO', '3'), 2751),
 (('CE', '4'), 2604),
 (('RS', '2'), 2549),
 (('AL', '3'), 2474),
 (('PR', '4'), 2399),
 (('PR', '2'), 2164),
 (('MT', '3'), 1823),
 (('SC', '4'), 1608),
 (('SE', '3'), 1572),
 (('RJ', '2'), 1547),
 (('TO', '3'), 1534),
 (('MA', '4'), 1394),
 (('SC', '2'), 1371),
 (('GO', '4'), 1334),
 (('MA', '2'), 1305),
 (('BA', '2'), 1168),
 (('PE', '2'), 1083),
 (('PA', '4'), 1046),
 (('GO', '2'), 1040),
 (('PB', '4'), 1022),
 (('AC', '3'), 1009),
 (('

In [62]:
rdd_somaQuant_ordenado.coalesce(1).saveAsTextFile("Quant-de-escolas-por-UF-e-DependenciaAdmin.txt")